<a href="https://colab.research.google.com/github/Tomo-for-lab/Code-of-automating-DE/blob/main/development%20of%20metaprompt/development_of_metaprompt_with_chat_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
!pip install -q openpyxl anthropic weave python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 381.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import ipykernel
from notebook.notebookapp import list_running_servers
import requests
from urllib.parse import urljoin
import json
import os
from google.colab import auth
from google.colab import userdata
from googleapiclient.discovery import build
import pandas as pd
import openpyxl
import base64
import csv
import numpy as np
import copy
import glob
import time
from docx import Document

from openai import OpenAI
from openai import AzureOpenAI
from tqdm import tqdm

Change the working directory to the path of the current notebook

In [ ]:
from google.colab import drive
# Mount Google Drive to access files stored in it
drive.mount('/content/drive')


def get_notebook_path():
    # Get the path of the current Jupyter notebook
    kernel_id = re.search('kernel-(.*).json', ipykernel.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

def get_folder_path(folder_id):
    # Recursively get the full path of a folder given its ID
    if folder_id:
        folder = drive_service.files().get(fileId=folder_id, fields="name, parents").execute()
        folder_name = folder.get('name')
        parents = folder.get('parents')
        if parents:
            parent_path = get_folder_path(parents[0])
            return parent_path + '/' + folder_name
        else:
            return folder_name
    return ''

def get_file_path(file_id):
    # Recursively get the full path of a file given its ID
    file = drive_service.files().get(fileId=file_id, fields="name, parents").execute()
    file_name = file.get('name')
    parents = file.get('parents')
    if parents:
        parent_id = parents[0]
        parent_path = get_folder_path(parent_id)
        return parent_path
    else:
        return file_name

# Get the path of the current notebook
notebook_path = get_notebook_path()


# Authenticate and initialize the Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Extract the file ID from the notebook path (assumes file ID is part of the path)
file_id = re.search(r'fileId=(\w+)', notebook_path).group(1)

# Get the full path of the file using its ID
file_path = get_file_path(file_id)
if 'マイドライブ' in file_path:
    converted_path = re.sub(r'(^|/)マイドライブ($|/)', '/content/drive/MyDrive/', file_path)
elif 'MyDrive' in file_path:
    converted_path = re.sub(r'(^|/)MyDrive($|/)', '/content/drive/MyDrive/', file_path)
else:
    converted_path = '/content/drive/MyDrive/' + file_path

# Change the working directory to the converted path
os.chdir(converted_path)

Mounted at /content/drive


 # To download original description

In [ ]:
# Path to the JSON file of original descriptions
json_file_path = 'original_description_insomnia.json'

# Load JSON data from the file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Extract all fields from the JSON data
all_fields = [field for field in data]

# Change type 'integer' to 'number' for tools
for item in all_fields:
    if item['type'] == 'integer':
        item['type'] = 'number'

In [ ]:
print(all_fields)

[{'name': 'Year', 'type': 'number', 'description': 'Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead.'}, {'name': 'ind_clu', 'type': 'string', 'description': "Study design: Choose 'individual' if individually randomized, choose 'cluster' if it is a cluster RCT. You have to choose from the options [individual, cluster]."}, {'name': 'ICC_for_cRCT', 'type': 'number', 'description': "Intra-cluster Correlation Coefficient for cluster RCTs. If it is not clearly stated, use '0.05'."}, {'name': 'Country', 'type': 'string', 'description': 'Countries where the trial took place.'}, {'name': 'Single_multi', 'type': 'string', 'description': 'Whether the recruitment took place at a single or multi centers. You have to choose from the options [single, multi].'}, {'name': 'Age_mean', 'type': 'number', 'description': 'Mean age of the patients in the arm.'}, {'name': 'Age_sd', 'type': 'number', 'description': 'Stand

# To download training data

In [ ]:
# Initialize an empty list to store the list of RCTs for training data.
sample_trials = []

# Open the file containing the list of training data
with open('sample_trials_list.txt', 'r') as file:
    # Read each line in the file
    for line in file:
        # Parse the JSON string on the current line and add it to the list
        sample_trials.append(json.loads(line.strip()))

# Preparation of Azure for GPT

In [ ]:
#deploy names
deployment_ids = ["gpt-4o","gpt-4-turbo","gpt-3.5-turbo","text-embedding-ada-002"]

#If you use Azure
#key
API_KEY = userdata.get('Azure_API_key')
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')
AZURE_API_version = userdata.get('AZURE_API_version')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=AZURE_API_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)


# Preparation of weave

Weave is the application to store interactions with LLMs as logs.

In [ ]:
import weave

# Initialize weave with a specific identifier
weave.init("hello-weave")

# Define required function

## Function to convert text data in JSON and table data into string variable.

In [ ]:
def extract_sentence(pdf_name):
  # Construct the path to the JSON file associated with the given PDF
  json_name=pdf_name+"/structuredData.json"

  a = open(json_name)# Open and load the JSON file
  j = json.load(a)
  method_flag=1
  extracted_sentence=""# Initialize a string to store the extracted sentences
  previous_item_class=0# To track the class of the previous item and determine if a newline is needed

  # Iterate through each element in the JSON file
  for item in j["elements"] :
    # Check if the item is a heading
    if ("//Document/H" in item['Path']) or (re.search(r"//Document/Sect.*/H", item['Path'])):
      if previous_item_class!=1:
        extracted_sentence+="\n"
        previous_item_class=1
      if "Text" in item:
        if ("reference" == item["Text"].strip().lower()) or ("references" == item["Text"].strip().lower()) or ("appendix" == item["Text"].strip().lower()):
          extracted_sentence+="\n"
          break;
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a table and convert table data to text
    elif (re.search("//Document/Table.*" ,item['Path'])) or (re.search(r"//Document/Sect.*/Table", item['Path'])):
      if previous_item_class!=2:
        extracted_sentence+="\n"
        previous_item_class=2
      if "filePaths" in item:
        for t in item["filePaths"]:
          if ".xlsx" in t:
            extracted_sentence+=t.replace(".xlsx", "").split("/")[-1]
            extracted_sentence+="\n"
            t=pdf_name+"/"+t
            # Load the Excel file
            df_t = pd.read_excel(t)
            # Remove carriage returns from column names and data
            df_t.columns = [col.replace('_x000D_', '') for col in df_t.columns]
            df_t = df_t.replace('_x000D_', '', regex=True)
            # Get column names as a string
            column_names = ' '.join(df_t.columns)

            extracted_sentence+=df_t.to_csv(index=False)
            extracted_sentence+="\n"

    # Check if the item is a footnote
    elif (re.search("//Document/Footnote\.*",item['Path'])) or (re.search(r"//Document/Sect.*/Footnote", item['Path'])):
      if previous_item_class!=5:
        extracted_sentence+="\n"
        previous_item_class=5
      if "Text" in item:
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a paragraph
    elif ("//Document/P" in item['Path']) or (re.search(r"//Document/Sect.*/P", item['Path'])):
      if previous_item_class!=3:
        extracted_sentence+="\n"
        previous_item_class=3
      if not("/Figure" in item["Path"]):
        if "Text" in item:
          extracted_sentence+=item["Text"]

    # Check if the item is a list
    elif ("//Document/L" in item['Path']) or (re.search(r"//Document/Sect.*/L", item['Path'])):
      if previous_item_class!=4:
        extracted_sentence+="\n"
        previous_item_class=4
      if "/Lbl" in item["Path"]:
        extracted_sentence+="\n"
      if "Text" in item:
        extracted_sentence+=item["Text"]


  # Close the JSON file
  a.close()
  return extracted_sentence

## Function to convert text data in JSON into string variable.

In [ ]:
def extract_sentence_text(pdf_name):
  # Construct the path to the JSON file associated with the given PDF
  json_name=pdf_name+"/structuredData.json"

  a = open(json_name)# Open and load the JSON file
  j = json.load(a)
  method_flag=1
  extracted_sentence=""# Initialize a string to store the extracted sentences
  previous_item_class=0 # To track the class of the previous item and determine if a newline is needed

  # Iterate through each element in the JSON file
  for item in j["elements"] :
    # Check if the item is a heading
    if ("//Document/H" in item['Path']) or (re.search(r"//Document/Sect.*/H", item['Path'])):
      if previous_item_class!=1:
        extracted_sentence+="\n"
        previous_item_class=1
      if "Text" in item:
        # Check for specific keywords to stop extracting sentences
        if ("reference" == item["Text"].strip().lower()) or ("references" == item["Text"].strip().lower()) or ("appendix" == item["Text"].strip().lower()):
          extracted_sentence+="\n"
          break;
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a paragraph
    elif ("//Document/P" in item['Path']) or (re.search(r"//Document/Sect.*/P", item['Path'])):
      if previous_item_class!=3:
        extracted_sentence+="\n"
        previous_item_class=3
      if not("/Figure" in item["Path"]):
        if "Text" in item:
          extracted_sentence+=item["Text"]

    # Check if the item is a list
    elif ("//Document/L" in item['Path']) or (re.search(r"//Document/Sect.*/L", item['Path'])):
      if previous_item_class!=4:
        extracted_sentence+="\n"
        previous_item_class=4
      if "/Lbl" in item["Path"]:
        extracted_sentence+="\n"
      if "Text" in item:
        extracted_sentence+=item["Text"]

    # Check if the item is a footnote
    elif (re.search("//Document/Footnote\.*",item['Path'])) or (re.search(r"//Document/Sect.*/Footnote", item['Path'])):
      if previous_item_class != 5:
        extracted_sentence+="\n"
        previous_item_class=5
      if "Text" in item:
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

  # Close the JSON file
  a.close()
  return extracted_sentence

## Function to create tools for development of metaprompt

In [ ]:
def make_tools_optimization(optimized_all_fields):
  # Create a dictionary with item names as keys and the rest of the item attributes as values
  properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in optimized_all_fields}

  # Create a list of names from the optimized fields
  name_list = [item['name'] for item in optimized_all_fields]


  # Define the base structure for the description tools
  description_tools = [{'type': 'function',
      'function':{
          'name': 'optimize_descriptions_of_study_features',
          'description': """      Follow the instructions below to optimize the descriptions in 'optimize_descriptions_of_study_features' tools.
      # Tool Updates
      When updating the description of tools, use the data extracted by you and correct answer data to be extracted. Provide as much detail as possible and include few-shot examples.
      # Tool Update Instructions:
      1. **Updating Tool Descriptions**:
      - Use data from papers, the data extracted by you and correct answer data to be extracted to update the tool descriptions.
        - When new tools are added, provide detailed explanations of their use and purpose.
        - Offer clear and detailed descriptions of each tool's functionality and how to use them.
      2. **Providing Few-Shot Examples**:
        - Offer concrete few-shot examples to demonstrate the use of new or existing tools.
        - Use examples to show the precise usage of the tools, making it easy for you to understand.
        """,
          'parameters': {
              'type': 'object',
            'properties': {}
          }
      }
  }
  ]

  # Copy the name list to a new list for use in the tool description
  splitted_name_list = copy.deepcopy(name_list)

  # Add the required names to the function parameters
  description_tools[0]['function']['parameters']['required'] = splitted_name_list

  # Add the properties for each name to the function parameters
  description_tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in splitted_name_list if key in properties_dict}

  return description_tools

## Function to encode images into data URL for input to GPT

In [ ]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

## Function for GPT function calling

In [ ]:
@weave.op() # 🐝
def GPT_function_calling(tools, messages, tools_name):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        tools=tools,
        tool_choice =  {"type": "function", "function": {"name": tools_name}},
        temperature=0
    )
  return response

## Function to update descripions for each variable.

In [ ]:
def optimizing_all_fields(optimized_all_fields, response, splitter):
  # Parse the response to get the output dictionary from the tool's function call
  output_dict = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

  # Iterate over the key-value pairs in the output dictionary
  for key, value in output_dict.items():

    # Find the matching item in the optimized_all_fields list
    for i in range(len(optimized_all_fields)):
      if optimized_all_fields[i]["name"] == key:
        # Check if the value is not an integer or float
        if not isinstance(value, (int, float)):
          # Update the description if the value is not "Not specified" or "*"
          if (str(value) != "Not specified") and (str(value) != "*"):
            optimized_all_fields[i]["description"] = value

  return optimized_all_fields

## Function to log the optimized description.

In [ ]:
def optimized_description_log(shot_name, fold_index, n, n_index, f_index, log_file_name, optimized_all_fields):

  # Construct the file path using the provided parameters
  file_name = f"{log_file_name}/{shot_name}/{n[n_index]}_paper/{fold_index}_fold/optimized_all_fields_{f_index}.json"

  # Write the optimized_all_fields data to a JSON file
  with open(file_name, 'w') as file:
      json.dump(optimized_all_fields, file, indent=4)

  # Print a completion message
  print("Log has been successfully written to 'log_description'")

##Function to log error

In [ ]:
def error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e):
  # Construct the file path for the error log
  error_log_file_name = f"{log_file_name}/error_log"

  # Append error details to the log file
  with open(error_log_file_name, 'a') as file:
    file.write(f"shot_name: {shot_name} \n")
    file.write(f"fold_index: {fold_index} \n")
    file.write(f"n: {n[n_index]} \n")
    file.write(f"f_index: {f_index} \n")
    file.write(str(e)+"\n")
    file.write("\n")

# Development of metaprompt

## What LLMs should be aware of when extracting data (free text)

In [ ]:
attention_query = """
#####Attention######

# From which paper to extract data
The primary paper is named AuthorYYYY, the registry ULR AuthorYYYY_NCT1234, the protocol AuthorYYYY_pr, the appendix/supplement files AuthorYYY_app or AuthorYYY_supp, and the secondary papers AuthorYYYY_s1,2,…
You can mainly extract data from primary paper.

#Mixed model repeated measures
- When to be careful
"mixed model", "mixed model repeated measures (MMRM)", "linear mixed model", "growth curve analysis ", "hierarchical model", etc.
If SEs are reported instead of SDs, be especially careful, as MMRM is highly likely.
- What to look out for in evaluation and data extraction
MMRM is said to be a method to estimate the effect of missing values of repeatedly measured outcomes without bias by making the assumption of missing at random
So, N (e.g., Severity_ep_n) in the extraction on outcomes should be the N that was analyzed (=in principle, number randomized; number analyzed for statistical analysis excluding early dropouts). Note that this is different from the number of people for whom outcome measures were available. (In the Christensen2016 example, 1149 patients were randomly assigned and 581 were available at 6 weeks for primary outcome measurement. (N should also be number randomized when calculating SD using SD=SE*sqrt(N).


# How to extract data for each arm
First, extract the data for each arm.
If the arm is further divided into several groups, merge these groups and calculate the integrated data.
If you cannot extract data by arm, extract data for all participants.
If you cannot find any data to extract or you do not have to extract data, output "-1".

# Which measure data you will take
You have to extract data (mean, SD, number, etc.) in the measure you previously extracted.

# The measure abbreviations
ISI: Insomnia Severity Index
SCI: Sleep Condition Indicator
FOSQ: Functional Outcomes of Sleep Questionnaire
ESS: Epworth Sleepiness Scale
PSQI: Pittsburgh Sleep Quality Index
AIS: Athens Insomnia Scale
WASO: Wake After Sleep Onset
SOL: Sleep Onset Latency

# How to calculate SD
You have to calculate SD and output as number (do not output as formula).
SD=SE*sqrt(n)
　If mixed model repeated measures were used, n is the number of people randomized or included in the analysis.
SD=(upper limit of [100-x]%CI - mean)*sqrt(n)/ NORM.S.INV([100-x/2]/100)
  If 90%CI, then ((upper limit of 90%CI - mean)*sqrt(n)/ norm.s.inv(0.95))
If 95%CI, then ((upper limit of 95%CI - mean)*sqrt(n)/ norm.s.inv(0.975))
If 99%CI, then ((upper limit of 99%CI - mean)*sqrt(n)/ norm.s.inv(0.995))
SD= (Q3 − Q1)/1.35
"""

## Optimize

In [ ]:
# Load the Excel file for original extracted data
file_path = 'DE.xlsx'
df = pd.read_excel(file_path,skiprows=1)

n=[1,2,3,4,5]

In [ ]:
shot_name = "chat"

weave.init('optimizing_description_integrate') # 🐝

In [ ]:
# Initialize variables
log_file_name = "optimized_description_log_v2"

# Iterate over the number of training data and folds
for n_index in range(5):
  for fold_index in range(10):
    # Path to the JSON file
    json_file_path = 'original_description_insomnia.json'

    # Load JSON data from the file
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Extract all fields from the JSON data
    all_fields = [field for field in data]
    _optimized_all_fields = copy.deepcopy(all_fields)
    png_files_sum = 0 # Sum of PNG files processed

    # Re-define tools with the optimized fields
    all_tools = make_tools_optimization(_optimized_all_fields)

    # System content for GPT model
    system_content = """
    You are a systematic reviewer and you have to complete the data extraction protocol.
    Now, I will present a research paper, the data extracted by you and correct answer data to be extracted so that you can optimize the value of “description” of each item to improve your future extractions.
    Please optimize the value of “description” of each item in the tool.\n
    Output only each "description" of each item in tools.
    Do not extract data from the paper.
    """

    # Query for GPT model
    query_GPT = """
    Follow the instructions below to optimize the descriptions in 'optimize_descriptions_of_study_features' tools.

    ## Tool Updates
    When updating the description of tools, use detailed information from papers, the data extracted by you and correct answer data to be extracted. Provide as much detail as possible and include few-shot examples when necessary.

    ### Tool Update Instructions:
    1. **Updating Tool Descriptions**:
      - Use data from papers, the data extracted by you and correct answer data to be extracted to update the tool descriptions.
      - When new tools are added, provide detailed explanations of their use and purpose.
      - Offer clear and detailed descriptions of each tool's functionality and how to use them.

    2. **Providing Few-Shot Examples**:
      - Offer concrete few-shot examples to demonstrate the use of new or existing tools.
      - Use examples to show the precise usage of the tools, making it easy for users to understand.
    """

    messages_GPT = []

    messages_GPT.append({"role": "system", "content": system_content})

    query_GPT += "The 'optimize_descriptions_of_study_features' tools:"
    query_GPT += json.dumps(all_tools)

    # Append user and assistant messages for development of metaprompt
    messages_GPT.append({"role": "user", "content": query_GPT})
    messages_GPT.append({"role": "assistant", "content": "According to the given paper, I will compare the extraction results with the correct data, optimize the description of the tool for data extraction, and give an example of data extraction. What is the paper, the extraction results, and the correct data?"})

    for f_index in range(n[n_index]):
      print("data extraction")
      f_name=sample_trials[n_index][fold_index][f_index]
      print("n_index: ", n_index)
      print("fold_index: ", fold_index)
      print("f_index: ", f_index)

      ############################################
      # Data extraction
      ############################################
      extracted_data = {} # For storing extracted data
      output_extracted_data = {} # For storing output data
      ############################################
      # Define tools for extracting arm information
      tools = [{'type': 'function',
          'function':
              {'name': 'extract_arm',
              'description': 'Extract the number of the arms and each arm name',
              'parameters': {
                  'type': 'object',
                'properties': {
                    "arms_num":
                    {"type": "number",
                    "description": "Answer the number of the arms"
                    },
                    "arm_name_1":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_2":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_3":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_4":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_5":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    }
                },
              "required": [
                  "arms_num",
                  "arm_name_1",
                  "arm_name_2",
                  "arm_name_3",
                  "arm_name_4",
                  "arm_name_5"
                  ]
              }
          }
      }
      ]
      ############################################
      # Create messages for extracting arm information
      messages_arm=[]
      system_content_arm = """You are a systematic reviewer.You have to extract arms(i.e., intervenitons and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.\n\n"""
      user_content_arm = """You have to extract arms(i.e., intervenitons and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.\n\n"""

      # Create input text from the paper
      root_dir = f"IncludedTrials/{f_name}" # Root directory containing the trial files
      file_to_find = "structuredData.json" # JSON file to find in directories

      user_content_arm += f"paper_name: {f_name} \n"
      for dirpath, dirnames, filenames in os.walk(root_dir):# Process each document in the directory
        if (dirpath == root_dir) or (dirpath == root_dir+f"/{f_name}"): # Only main documents
          if file_to_find in filenames:
            main_paper_txt= extract_sentence_text(dirpath)
            user_content_arm += main_paper_txt

      messages_arm.append({"role": "system", "content": system_content_arm})
      messages_arm.append({"role": "user", "content": user_content_arm})

      ############################################
      # Get response from GPT model
      try:
        tools_name = "extract_arm"
        response_arm = GPT_function_calling(tools, messages_arm, tools_name) # Call GPT function for arm extraction
      except Exception as e:
        error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e)
        error_message = f"An error occurred response arm: {str(e)}"
        print(error_message)
        continue;

      ############################################
      # Parse the response
      try:
        output_dict_arm = json.loads(response_arm.choices[0].message.tool_calls[0].function.arguments)# Parse response
      except Exception as e:
        error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e)
        error_message = f"An error occurred output_dict_arm : {str(e)}"
        print(error_message)
        continue;

      arms_num = output_dict_arm["arms_num"]# Number of arms
      arms_list = []# List of arm names
      for arm_num in range(arms_num):
        arms_list.append(output_dict_arm[f"arm_name_{arm_num+1}"])
      print("arms_num: ", arms_num)
      print("arms_list: ", arms_list)

      ############################################
      # Process each arm
      for arm_num in range(arms_num):
        print("arm_num: ", arm_num)

        for split_num in range(2):
          ############################################
          # Create tools for optimized description
          properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in _optimized_all_fields}
          name_list = [item['name'] for item in _optimized_all_fields]

          description_tools = [{'type': 'function',
              'function':
                  {'name': 'extract_study_features',
                  'description': 'Extracts key features from a user-inputted research article for systematic review purposes.',
                  'parameters': {
                      'type': 'object',
                    'properties': {}
                  }
              }
          }
          ]

          # Split the name list for processing
          if split_num==0:
            splitted_name_list = copy.deepcopy(name_list[:30])
          else:
            splitted_name_list = copy.deepcopy(name_list[30:len(name_list)])

          description_tools[0]['function']['parameters']['required'] = splitted_name_list
          description_tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in splitted_name_list if key in properties_dict}

          ############################################
          # Create messages for data extraction
          system_content_DE = """You are a systematic reviewer.
          Use the extract_study_features tool in your response.
          """

          query_GPT = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
            Use the extract_study_features tool in your response.\n
              """
          query_GPT += attention_query
          query_GPT += f"trial_name: {f_name} \n \n "

          query_function_calling = copy.deepcopy(query_GPT)

          query_GPT += "The extract_study_features:"
          query_GPT += json.dumps(description_tools)

          last_query = attention_query

          content_prompt_GPT_DE = [] # List to hold content prompts for GPT

          png_files_sum_DE = 0 # Sum of PNG files processed

          # Create input text from the paper
          root_dir = f"IncludedTrials/{f_name}" # Root directory containing the trial files
          file_to_find = "structuredData.json" # JSON file to find in directories

          content_prompt_GPT_DE.append({"type": "text", "text": query_GPT})

          # Extract sentences from each document in the directory
          for dirpath, dirnames, filenames in os.walk(root_dir):
            if f_name in dirnames:# Process the main paper first
              dirnames.remove(f_name)
              dirnames.insert(0, f_name)
            if file_to_find in filenames:
              paper_query = dirpath.split("/")[-1]
              paper_query += extract_sentence(dirpath)
              content_prompt_GPT_DE.append({"type": "text", "text":paper_query})

              # Filter and add PNG files to the content prompt
              if os.path.exists(f"{dirpath}/figures"):
                png_files = [file for file in os.listdir(f"{dirpath}/figures") if file.endswith('.png')]

                for i in range(len(png_files)):
                  if png_files_sum_DE >= 10: # Limit the number of images to 10
                    break;
                  content_prompt_GPT_DE.append({"type": "image_url", "image_url": {"url": local_image_to_data_url(f"{dirpath}/figures/"+png_files[i])}})
                  png_files_sum_DE += 1

          # Process txt and docx files in the root directory
          txt_docx_query = "" # String to hold txt and docx file contents
          txt_docx_flag = False # Flag to check if txt or docx files are found

          for file_name in os.listdir(root_dir):
              if file_name.endswith('.txt'):
                  txt_docx_flag = True
                  txt_docx_query += "txt_name: "
                  txt_docx_query += "txt_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  with open(file_path, 'r', encoding='utf-8') as file:
                      txt_docx_query += file.read()
              elif file_name.endswith('.docx'):
                  txt_docx_flag = True
                  txt_docx_query += "docx_name: "
                  txt_docx_query += "docx_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  doc = Document(file_path)
                  full_text = []
                  for para in doc.paragraphs:
                      full_text.append(para.text)

                  for table in doc.tables:# Read tables in the document
                      for row in table.rows:
                          for cell in row.cells:
                              full_text.append(cell.text)
                  txt_docx_query += '\n'.join(full_text)

          if txt_docx_flag:
            content_prompt_GPT_DE.append({"type": "text", "text":txt_docx_query}) # Add txt/docx content if found


          content_prompt_GPT_DE.append({"type": "text", "text": last_query}) # Add the last query to the content prompt
          messages_GPT_DE = []
          messages_GPT_DE.append({"role": "system", "content": system_content_DE})
          messages_GPT_DE.append({"role": "user", "content": content_prompt_GPT_DE})

          ############################################
          try:
            # Get response from GPT model
            tools_name = "extract_study_features" # Name of the tool being used
            response_GPT = GPT_function_calling(description_tools, messages_GPT_DE ,tools_name) # Call GPT function for study features extraction
          except Exception as e:
            extracted_data[arms_list[arm_num]] = copy.deepcopy(output) # Copy output to extracted data in case of error
            error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e) # Log the error
            error_message = f"An error occurred data extraction: {str(e)}"
            print(error_message)
            continue;
          ############################################
          # Store the results
          output_dict_GPT = json.loads(response_GPT.choices[0].message.tool_calls[0].function.arguments)
          for k in output_dict_GPT.keys():
            if k in output.keys():
              output[k] = output_dict_GPT[k] # Update output with GPT response
          extracted_data[arms_list[arm_num]] = copy.deepcopy(output) # Store the extracted data for the current arm
        print(extracted_data[arms_list[arm_num]])

############################################
#Optimization
############################################

      for split_num in range(2):
        # Split the fields for processing: first 30 and the rest
        # Create messages for optimization
        if split_num == 0:
          optimized_all_fields = copy.deepcopy(_optimized_all_fields[:30])
        else:
          optimized_all_fields = copy.deepcopy(_optimized_all_fields[30:len(_optimized_all_fields)])

        if split_num == 0:
          # Define root directory and file to find
          root_dir = f"IncludedTrials/{f_name}" # Root directory containing the RCT files
          file_to_find = "structuredData.json" # JSON file to find in directories

          content_prompt_GPT=[] # List to hold prompts for GPT model
          first_flag = True # Flag to process the main paper first

          # Traverse through directories and process files
          for dirpath, dirnames, filenames in os.walk(root_dir):
            if f_name in dirnames:# Process the main paper first
              dirnames.remove(f_name)
              dirnames.insert(0, f_name)
            if file_to_find in filenames:
              if first_flag:
                paper_query = f"<example {f_index+1}> \n"
                paper_query += f"trial_name: {f_name} \n"
                paper_query += dirpath.split("/")[-1]
                first_flag = False
              else:
                paper_query = dirpath.split("/")[-1]
              paper_query += extract_sentence(dirpath)
              content_prompt_GPT.append({"type": "text", "text":paper_query})

              # Filter and add PNG files to the content prompt
              if os.path.exists(f"{dirpath}/figures"):
                png_files = [file for file in os.listdir(f"{dirpath}/figures") if file.endswith('.png')]

                for i in range(len(png_files)):
                  if png_files_sum >= 10: # Limit the number of images to 10
                    break;
                  content_prompt_GPT.append({"type": "image_url", "image_url": {"url": local_image_to_data_url(f"{dirpath}/figures/"+png_files[i])}})
                  png_files_sum += 1

          # Process txt and docx files in the root directory
          txt_docx_query = "" # String to hold txt and docx file contents
          txt_docx_flag = False # Flag to check if txt or docx files are found
          for file_name in os.listdir(root_dir):
              if file_name.endswith('.txt'):
                  txt_docx_flag = True
                  txt_docx_query += "txt_name: "
                  txt_docx_query += "txt_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  with open(file_path, 'r', encoding='utf-8') as file:
                      txt_docx_query += file.read()
              elif file_name.endswith('.docx'):
                  txt_docx_flag = True
                  txt_docx_query += "docx_name: "
                  txt_docx_query += "docx_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  doc = Document(file_path)
                  full_text = []
                  for para in doc.paragraphs:
                      full_text.append(para.text)
                  for table in doc.tables:
                      for row in table.rows:
                          for cell in row.cells:
                              full_text.append(cell.text)
                  txt_docx_query += '\n'.join(full_text)
          if txt_docx_flag:
            content_prompt_GPT.append({"type": "text", "text":txt_docx_query})

          # Create query for extracted and correct data
          answer_query = f"\n <The extracted data for example {f_index+1}> \n"
          answer_query += "The data extracted by you are following: \n"
          arm_num=0
          for extracted_arm_name in extracted_data:
            answer_query += f"arm_{arm_num}: "
            answer_query += extracted_arm_name
            answer_query += "\n"
            for key, value in extracted_data[extracted_arm_name].items():
              answer_query += key
              answer_query += ": "
              answer_query += str(value)
              answer_query += "\n"
            answer_query += "\n"
            arm_num+=1
          answer_query += "\n"

          answer_query += "Correct answer data to be extracted are following: \n"
          pdf_df = df[df["study"]==f_name] # DataFrame for the study

          for arm_num in range(len(pdf_df)):
            answer_query += f"arm_{arm_num}: "
            answer_query += pdf_df.iloc[arm_num]["Arm"]
            answer_query += "\n"
            for field_num in range(len(all_fields)):
              answer_query += all_fields[field_num]["name"]
              answer_query += ": "
              answer_query += str(pdf_df.iloc[arm_num][all_fields[field_num]["name"]])
              answer_query += "\n"
            answer_query += "\n"
          answer_query += "\n"

          content_prompt_GPT.append({"type": "text", "text": answer_query})

          messages_GPT.append({"role": "user", "content": content_prompt_GPT}) # Add to GPT messages

        ############################################
        # Optimize descriptions
        try:
          tools = make_tools_optimization(optimized_all_fields) # Re-define tools
          for name in tools[0]['function']['parameters']['properties']:
            tools[0]['function']['parameters']['properties'][name]['description'] = ' You have to optimize the description below: ' + tools[0]['function']['parameters']['properties'][name]['description']

          tools_name = "optimize_descriptions_of_study_features" # Tool name for GPT
          response_GPT = GPT_function_calling(tools, messages_GPT, tools_name) # Get response from GPT
          response_error_flag = False # Initialize error flag

        except Exception as e:
          response_error_flag = True
          messages_GPT.append({"role": "assistant", "content": json.dumps(_optimized_all_fields)})
          print(f_name+":"+json.dumps(_optimized_all_fields))
          optimized_description_log(shot_name, fold_index, n, n_index,f_index,log_file_name,_optimized_all_fields)
          error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e)
          print("error occured: ", str(e))
          break;
        if not(response_error_flag):
          try:
            _optimized_all_fields = optimizing_all_fields(_optimized_all_fields, response_GPT, "\n") # Optimize descriptions for all variables
            print(response_GPT)
          except Exception as e:
            error_log(shot_name, fold_index, n, n_index,f_index,log_file_name,e)
            print("error occured", str(e))
            continue;
      if response_error_flag:
        continue;
      messages_GPT.append({"role": "assistant", "content": json.dumps(_optimized_all_fields)}) # Add optimized descriptions (which means the response of GPT) to messages
      print(f_name+":"+json.dumps(_optimized_all_fields))
      optimized_description_log(shot_name, fold_index, n, n_index,f_index,log_file_name,_optimized_all_fields) # Log the optimized descriptions

data extraction
n_index:  0
fold_index:  0
f_index:  0
🍩 https://wandb.ai/srws-psg/optimizing_description_integrate/r/call/43ccdbaa-9acd-4817-8f99-9ae10d7878e2
arms_num:  2
arms_list:  ['psychoeducational intervention (PI+)', 'control (PI-)']
arm_num:  0
🍩 https://wandb.ai/srws-psg/optimizing_description_integrate/r/call/c1389ef0-35ec-4fed-a325-daf63987a8fb
🍩 https://wandb.ai/srws-psg/optimizing_description_integrate/r/call/72af32a6-8367-4931-bb24-fb26608178ba
{'Year': 2017, 'ind_clu': 'individual', 'ICC_for_cRCT': 0.05, 'Country': 'Italy', 'Single_multi': 'single', 'Age_mean': 64.42, 'Age_sd': 12.94, 'Age_n': 19, 'F_n': 10, 'Hyp_n': 0, 'Insomnia_diagnosis': '1. formal_ICSD', 'Primary_insomnia_n': 19, 'Comorbidities': '0. primary_insomnia', 'se': 1, 'sd': 1, 'cr': 0, 'cw': 0, 'sr': 0, 'sc': 0, 're': 0, 'pi': 0, 'w': 0, 'dt': 0, 'ns': 0, 'he': 0, 'tg': 1, 'ind': 1, 'gp': 0, 'ff': 1, 'ae': 0, 'n': 19, 'remission': -1, 'def_remission_response': '-1', 'scale_used': 'ISI', 'Severity_bl_mean

Exception in thread Thread-18 (_process_batches):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace_server/async_batch_processor.py", line 61, in _process_batches
    self.processor_fn(current_batch)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 330, in wrapped_f
    return self(f, *args, **kw)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 467, in __call__
    do = self.iter(retry_state=retry_state)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 368, in iter
    result = action(retry_state)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 390, in <lambda>
    self._add_action_func(lambda rs: rs.outcome.result())
  File "/usr/lib/python3

🍩 https://wandb.ai/srws-psg/optimizing_description_integrate/r/call/e939256c-adc4-47d4-af6e-994d82b71ac7
🍩 https://wandb.ai/srws-psg/optimizing_description_integrate/r/call/11b1a94b-d663-4536-8134-3ef2f29107de
{'Year': 2014, 'ind_clu': 'individual', 'ICC_for_cRCT': 0.05, 'Country': 'Canada', 'Single_multi': 'single', 'Age_mean': 58.73, 'Age_sd': 10.46, 'Age_n': 40, 'F_n': 32, 'Hyp_n': 40, 'Insomnia_diagnosis': '1. formal_ICSD', 'Primary_insomnia_n': 40, 'Comorbidities': '1. primary_secondary_mixed', 'se': 1, 'sd': 1, 'cr': 1, 'cw': 1, 'sr': 1, 'sc': 1, 're': 1, 'pi': 0, 'w': 0, 'dt': 0, 'ns': 0, 'he': 1, 'tg': 1, 'ind': 0, 'gp': 1, 'ff': 1, 'ae': 0, 'n': 47, 'remission': -1, 'def_remission_response': '-1', 'scale_used': 'ISI', 'Severity_bl_mean': 18.25, 'Severity_bl_sd': 4.18, 'Severity_bl_n': 47, 'Severity_ch_mean': -1, 'Severity_ch_sd': -1, 'Severity_ch_n': -1, 'Severity_ep_mean': 8.2, 'Severity_ep_sd': 3.88, 'Severity_ep_n': 47, 'Severity_scale': 'ISI', 'dropout': 10, 'SE_mean': 84.

#For 0_RCT for training

In this case, the optimized descriptions will be the original descriptions.

In [ ]:
# Initialize variables
shot_name = "chat" # Name of the shot for logging purposes
log_file_name = "optimized_description_log_v2" # Name of the log file
n=[0] # List with a single element for indexing
n_index=0 # Index for the list n

# JSON file path
json_file_path = 'original_description_insomnia.json'

# Load JSON data from the file
with open(json_file_path, 'r') as file:
    data = json.load(file) # Load data from JSON file

# Extract descriptions of all variables from the original description JSON data.
all_fields = [field for field in data]
for fold_index in range(10):
  f_index=0 # Initialize field index
  # Log the optimized descriptions for each fold
  optimized_description_log(shot_name, fold_index, n, n_index,f_index,log_file_name, all_fields)


Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
Log has been successfully written to 'log_description'
